## 2. 기본 개념들과 베이지안 확률론적 관점

본 장에서는, 조건부 확률에 대한 개본적인 개념을 소개하고, 베이즈 이론에 대해 설명할 것입니다. 이산 사건들과 연속 변수 예측에 대한 다양한 예제를 다룰 것입니다. 그리고 베이지안 모델 갱신에 대해 설명할 것입니다. 모델 식별 가능성에 대한 문제도 다룰 것입니다. 결정론적 방법과 확률론적 방법을 비교할 것입니다. 확률론적 방법의 장점들, 어려운 점들, 개선이 필요한 사항들에 대해 이야기 할 것입니다. 선형 및 비선형 회기 문제에 대해 다룰 것입니다. 업데이트 한 확률밀도함수는 매우 복잡할 수 있기 때문에, 그 확률밀도함수를 표현할 수 있는 샘플을 생성하기 위한 적응 MCMC 시뮬레이션 방법을 소개할 것입니다. 마지막으로 베이즈 추론의 기본 개념을 설명하기 위한 3가지 적용 사례를 소개할 것입니다. 첫째로,  베이지안 방법을 사용해 건물의 고유진동수와 대기 온도 사이의 관계를 찾아낼 것입니다. 길이와 영수에 한 온도의 영향을 관찰해 보면, 건물이 빔으로 모델링 가능한 경우 온도와 고유진동수 사이의 관계는 2차함수 형태가 됩니다. 22층 건물에서 6개월간 측정한 데이터를 사용하여 베이지안 방법으로 관련 계수들 및 불확실성에 대해 추정할 것입니다. 두 번째 적용 사례는 잘 알려진 칼만필터 입니다. 칼만필터 또한 베이지안 업데이트 방법으로 볼수 있으며, 상태 추정 및 변수 식별에 유용합니다. 하지만 잡음 변수들은 일반적으로 사전 정보 또는 경험을 바탕으로 사용자가 부여합니다. 본 예제에서는 베이지안 접근법을 사용하여 이 변수들을 추정할 것입니다. 마지막 적용 사례는 $PM_{10}$ 농축에 대한 대기 공기 질 예측에 관한 것 입니다. 실시간 예측 시스템을 개발하고, 칼만필터를 사용해 시스템을 업데이트할 것입니다.  변수 모델을 적절히 선택하면, 이 예측 시스템은 잘 알려진 인공 신경망보다 더 좋은 성능을 보이게 됩니다. 특히 높은 $PM_{10}$ 농축 데이터를 예측하는 경우 성능이 더 좋게 됩니다.

## 2.1 조건부 확률 및 기본 개념

두 개의 사건 A, B가 있습니다. **조건부 확률**은 사건 B가 일어난 후 사건 A가 발생할 확률이며 다음과 같습니다.

$$P(A|B)=\frac{P(A \cap B)}{P(B)} \tag{2.1}$$

상기 식은 $P(B)>0$일 때만 성립하며, 만약 $P(B)=0$이면 $P(A \cap B)=0$이 되고, $P(A|B)$는 의미가 없게 됩니다. $\cap$ 기호는 논리 연산자 and를 의미하며, $P(A \cap B)$는 두개의 사건이 모두 발생할 확률을 의미합니다.